In [5]:
%pip install pandas
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd
stats = pd.read_csv('player_mvp_stats.csv')

In [18]:
stats = stats.fillna(0)
stats.columns

Index(['Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year',
       'Pts Won', 'Pts Max', 'Share', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS'],
      dtype='object')

## Filtering Numerical Columns for Predictor Variables

In this section, we will filter only the numerical columns from the dataset, excluding non-numeric columns that we don't need. Additionally, we'll remove columns such as 'MVP points' and 'Win Shares', as 'MVP points' directly influence 'Win Shares', which we are trying to predict.

### Steps:
1. Select only the numerical columns from the dataset.
2. Exclude 'MVP points' and 'Win Shares' from the predictors to avoid data leakage.
3. Prepare the predictor variables `X` for model training.


In [53]:
if 'Unnamed: 0' in stats.columns:
    del stats['Unnamed: 0']

predictors = ['Age','G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'Year', 'W', 'L', 'W/L%', 'GB', 'PS/G',
       'PA/G', 'SRS']

In [54]:
train = stats[stats['Year'] < 2021]
test = stats[stats['Year'] == 2021]

In [55]:
from sklearn.linear_model import Ridge

reg = Ridge(alpha=0.1)

In [56]:
reg.fit(train[predictors], train["Share"])

Ridge(alpha=0.1)

In [57]:
predictions = reg.predict(test[predictors])

In [58]:
predictions = pd.DataFrame(predictions, columns=['predictions'], index=test.index)

In [59]:
predictions

,predictions
630,0.013567
631,-0.013756
632,0.002414
633,-0.004421
634,0.010734
...,...
13897,-0.012571
13898,-0.011575
13899,0.016424
13900,-0.020434


In [60]:
combination = pd.concat([test[["Player", "Share"]], predictions], axis=1)

In [61]:
combination

,Player,Share,predictions
630,Aaron Gordon,0.0,0.013567
631,Austin Rivers,0.0,-0.013756
632,Bol Bol,0.0,0.002414
633,Facundo Campazzo,0.0,-0.004421
634,Greg Whittington,0.0,0.010734
...,...,...,...
13897,Patty Mills,0.0,-0.012571
13898,Quinndary Weatherspoon,0.0,-0.011575
13899,Rudy Gay,0.0,0.016424
13900,Tre Jones,0.0,-0.020434


In [62]:
combination.sort_values("Share", ascending=False)

,Player,Share,predictions
641,Nikola Jokić,0.961,0.154307
8624,Joel Embiid,0.580,0.162713
3651,Stephen Curry,0.449,0.142386
9907,Giannis Antetokounmpo,0.345,0.207436
1389,Chris Paul,0.138,0.072294
...,...,...,...
4171,Chris Chiozza,0.000,0.007047
4170,Bruce Brown,0.000,0.003656
4169,Blake Griffin,0.000,0.011746
4168,Andre Roberson,0.000,-0.026907
